In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os 
os.chdir('/content/drive/MyDrive/vietnamesesentiment ')

In [4]:
import pandas as pd 
df = pd.read_csv('combine.csv')
df.head()

,sentence,label
0,Tao hít không khí thui cũng mập địt 😂😂,1
1,Nguyễn Hồng DiệuÝ má chị hằng mập địt vler,0
2,Lê Viết Quang nghe rất địt mẹ mày,0
3,Thành Trần địt phải chả bạn ơi nó lấy lãy 50% ...,0
4,Vay của home còn đỡ Bọn f.e t đéo vay bgio mu...,0


In [5]:
df['label'].value_counts()

2    16822
0    15775
1      515
Name: label, dtype: int64

In [6]:
df.shape

(33112, 2)

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from xgboost.sklearn import XGBClassifier
from collections import Counter

import numpy as np
# import unidecode
import pandas as pd
import string
import re
import os
import shutil
import pickle 

In [8]:
# from underthesea.word_tokenize.regex_tokenize import tokenize

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from xgboost.sklearn import XGBClassifier
from collections import Counter

import numpy as np
# import unidecode
import pandas as pd
import string
import re
import os
import shutil
import pickle 

In [9]:
df = pd.read_csv('combine.csv')
X = df['sentence'].values
Y = df['label'].values
index = np.arange(X.shape[0])
np.random.shuffle(index)
X = X[index]
Y = Y[index]

In [27]:
df.describe

<bound method NDFrame.describe of                                                 sentence  label
0                 Tao hít không khí thui cũng mập địt 😂😂      1
1             Nguyễn Hồng DiệuÝ má chị hằng mập địt vler      0
2                      Lê Viết Quang nghe rất địt mẹ mày      0
3      Thành Trần địt phải chả bạn ơi nó lấy lãy 50% ...      0
4      Vay của home còn đỡ  Bọn f.e t đéo vay bgio mu...      0
...                                                  ...    ...
33107  cái ngân hàng đểu dịch vụ kém cỏi, ATM toàn lỗ...      0
33108  vừa rút tiền cây ATM Tân Sơn Nhì, cây báo giao...      0
33109  Rút tiền không được ở Chi nhánh kỳ đồng! 24/7 ...      0
33110                                    Vâng e cảm ơn ạ      2
33111  Chuần luôn mình cũng bị chiều nay, rút k thành...      0

[33112 rows x 2 columns]>

In [10]:
def transform(x):
  return [s.lower() for s in x]
# def remove_tone(x):
#   return [unidecode.unidecode(s) for s in x] 
def remove_duplicate(x):
  result = []
  for s in x:
      s = re.sub(r'([a-z])\1+', lambda m: m.group(1), s, flags=re.IGNORECASE)
      s = re.sub(r'([a-z][a-z])\1+', lambda m: m.group(1), s, flags=re.IGNORECASE)
      result.append(s)
  return result

In [11]:
def transform(x):
  return [s.lower() for s in x]
# def remove_tone(x):
#   return [unidecode.unidecode(s) for s in x] 
def remove_duplicate(x):
  result = []
  for s in x:
      s = re.sub(r'([a-z])\1+', lambda m: m.group(1), s, flags=re.IGNORECASE)
      s = re.sub(r'([a-z][a-z])\1+', lambda m: m.group(1), s, flags=re.IGNORECASE)
      result.append(s)
  return result

In [12]:
X = transform(X)
X = remove_duplicate(X)

In [15]:
X[0]

"< a clas = ' hashtag-link ' href = ' / ho-chi-minh/hashtag/fondant-188581 ' > # fondant </a> , < a clas = ' hashtag-link ' href = ' / ho-chi-minh/hashtag/banh-kem-fondant-188582 ' > # bánh kem fondant </a> , < a clas = ' hashtag-link ' href = ' / ho-chi-minh/hashtag/fondant-an-tuong-188583 ' > # fondant ấn tượng </a> ,"

In [14]:
X[1]

'không gian trên lầu hẹp do bàn ghế kê sát nhau quá . ở đây sẽ yên tĩnh nếu không có 1 đám nào đấy đánh bài làm mất trật tự . đồ uống mình thấy bình thường . kem thì mình k ăn nên k biết . wifi yếu'

In [16]:
X[2]

'vị trí mặt tiền , hình thức tốt , ngon , phục vụ thì chưa đánh giá được , còn về giá thì không phù hợp cho sinh viên rồi . tô nhỏ 40k lớn 50 đặc biệt 60k . mới ăn thử tô lớn được 1 con ghẹ , nên ăn nếu muốn thưởng thức ^ ^'

In [29]:
text = 'không gian trên lầu hẹp do bàn ghế kê sát nhau quá . ở đây sẽ yên tĩnh nếu không có 1 đám nào đấy đánh bài làm mất trật tự . đồ uống mình thấy bình thường . kem thì mình k ăn nên k biết . wifi yếu'
len(text.split())

50

In [25]:
X[1].dtype

dtype('float64')

In [17]:
vectorizer = TfidfVectorizer(ngram_range=(1,4),norm='l2', min_df=2,max_features=6000)
X = vectorizer.fit_transform(X)

In [18]:
X.shape

(33112, 6000)

In [20]:
X[1]

<1x6000 sparse matrix of type '<class 'numpy.float64'>'
	with 56 stored elements in Compressed Sparse Row format>

In [ ]:
SVM = SVC(kernel='linear', C=0.72, class_weight='balanced', verbose=True, probability=True)

In [ ]:
X_train = X[:-10000]
Y_train = Y[:-10000]
X_test = X[-10000:]
Y_test = Y[-10000:]

SVM.fit(X_train, Y_train)

train_acc = (SVM.predict(X_train) == Y_train).astype(int).mean()
test_acc = (SVM.predict(X_test) == Y_test).astype(int).mean()

print("Train acc: ", train_acc, " Test acc: ", test_acc)

[LibSVM]Train acc:  0.9039027345102112  Test acc:  0.863


In [ ]:
X_train = X[:-10000]
Y_train = Y[:-10000]
X_test = X[-10000:]
Y_test = Y[-10000:]

In [ ]:
X_train.shape

(23112, 6000)

In [ ]:
SVM2 = SVC(kernel='rbf', C=0.72, class_weight='balanced', verbose=True, probability=True)

In [ ]:
SVM2.fit(X_train, Y_train)

train_acc = (SVM2.predict(X_train) == Y_train).astype(int).mean()
test_acc = (SVM2.predict(X_test) == Y_test).astype(int).mean()

print("Train acc: ", train_acc, " Test acc: ", test_acc)

[LibSVM]Train acc:  0.9385600553824853  Test acc:  0.867
